
# Slim simulations



In [1]:
import re, os, sys, pickle, pickle, math
from pathlib import Path
import numpy as np
import pandas as pd
import scipy
from pandas import DataFrame, Series
from sklearn.decomposition import PCA

# my own libaries
from GenomicWindows import window
import GenomicIntervals

np.random.seed(7)

Import variables global to the entire analysis:

In [2]:
import analysis_globals as ag

Local code in the scripts dir on the cluster:

In [3]:
if str(ag.scripts_dir) not in sys.path:
    sys.path.append(str(ag.scripts_dir))

import simons_meta_data
import hg19_chrom_sizes

Plotting setup:

In [4]:
%matplotlib inline

# Make inline plots vector graphics instead of raster graphics
from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('pdf', 'svg')
set_matplotlib_formats('retina', 'png')

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
from mpl_toolkits.basemap import Basemap
#matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

import mpld3

import seaborn as sns
sns.set() # seaborn niceness
sns.set_style("whitegrid")
sns.set_context("notebook")

# lowess for plotting
from statsmodels.nonparametric.smoothers_lowess import lowess

Ignore warnings:

In [5]:
# silence deprecation warnings (lots from seaborn)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

## Load meta data

Easy loading of meta data in a consistent manner across code

In [6]:
individuals, populations, regions = simons_meta_data.get_meta_data(meta_data_dir=ag.meta_data_dir)

chromosome_lengths = dict((k.replace('chr', ''), v) for k, v in hg19_chrom_sizes.hg19_chrom_sizes.items())

## Load swept regions

In [7]:
extended_peak_regions_10 = pd.read_hdf(ag.results_dir / 'extended_peak_regions_10%.hdf')
extended_peak_regions_10['chrom'] = 'chrX'
(extended_peak_regions_10.end_pos - extended_peak_regions_10.start_pos).sum()

32500000

## TMRCA computations

In [8]:
from math import exp

def lamb(i):
    # lamb(i) rate of coalescence from i-1 to i
    i += 1
    return i*(i-1)/2

samples = 5 # total samples
setsize = 3 # size of clade (we want to see at least one such clade coalesce before time t)
t = 1

for n in range(setsize, samples+1):
    # loop from setsize to samples (3 to 5):
    p = 0
    for i in range(1, n):
        # loop from 1 to n-1:
        p += np.prod([lamb(j)/(lamb(j)-lamb(i)) for j in range(1, n) if i != j]) \
        * exp(-lamb(n)*t) \
        * (lamb(i)/(lamb(i)-lamb(n))) \
        * (-exp(-(lamb(i)-lamb(n))*t) + 1)
    # combinatorial expression:
    p *= scipy.misc.comb(n, setsize) / scipy.misc.comb(samples, setsize)
p
    


0.035592882340661172

In [9]:
def lamb(x):
    x += 1
    return x*(x-1)/2


N = 3000
n = 42
t = 1

prob = 0
for i in range(1, n):
    prob += (1 - math.exp(-lamb(i)*t)) * np.prod([lamb(j)/(lamb(j)-lamb(i)) for j in range(1, n) if i != j])
prob * scipy.misc.comb(140, 42)

1.5316066224579469e+35

In [10]:
scipy.misc.comb(140, 42)

1.0163968567466686e+36

In [11]:
1- math.exp(-lamb(1))

0.6321205588285577

In [12]:
def lamb(x):
    return x*(x-1)/2

t = 10
n = 20

prob = 0
for i in range(1, n+1):
    print([lamb(j)/(lamb(j)-lamb(i)) for j in range(1, n+1) if i != j])
    prob += lamb(i) * math.exp(-lamb(i)*t) * np.prod([lamb(j)/(lamb(j)-lamb(i)) for j in range(1, n+1) if i != j])
prob #* scipy.misc.comb(n, i)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[-0.0, 1.5, 1.2, 1.1111111111111112, 1.0714285714285714, 1.05, 1.037037037037037, 1.0285714285714285, 1.0227272727272727, 1.0185185185185186, 1.0153846153846153, 1.0129870129870129, 1.011111111111111, 1.0096153846153846, 1.0084033613445378, 1.0074074074074073, 1.006578947368421, 1.0058823529411764, 1.0052910052910053]
[-0.0, -0.5, 2.0, 1.4285714285714286, 1.25, 1.1666666666666667, 1.12, 1.0909090909090908, 1.0714285714285714, 1.0576923076923077, 1.0476190476190477, 1.04, 1.0340909090909092, 1.0294117647058822, 1.0256410256410255, 1.0225563909774436, 1.02, 1.0178571428571428, 1.0160427807486632]
[-0.0, -0.2, -1.0, 2.5, 1.6666666666666667, 1.4, 1.2727272727272727, 1.2, 1.1538461538461537, 1.1224489795918366, 1.1, 1.0833333333333333, 1.0705882352941176, 1.0606060606060606, 1.0526315789473684, 1.0461538461538462, 1.0408163265306123, 1.0363636363636364, 1.0326086956521738]
[-0.0, -0.1111111111111

0.0

In [13]:
lamb(3)

3.0

In [14]:
import scipy

def coal_n_to_l(n, l, t):
    """Prob of reducing n lineages to l in time t"""
    Q = np.zeros(shape=(n-l+1, n-l+1))
    for x in range(n-l):
        rate = scipy.misc.comb(n-x, 2) / (2 * pop_size)
        Q[x][x] = -rate
        Q[x][x+1] = rate
    return scipy.linalg.expm(Q*t)[0][-1]

def clade_prob(n, l, t, i):
    """Prob of one among l lineages being ancestor to i among n"""
#     if i == 1 or l == 1:
#         return 1
#     else:
    return scipy.misc.comb(n-i-1, l-2) / scipy.misc.comb(n-1, l-1)
        
n = 5
t = 36
pop_size = 300
i = 2

total_prop = 0
for l in range(1, n):
    total_prop += coal_n_to_l(n, l, t) * clade_prob(n, l, t, i)
total_prop

0.1403635580926885

In [15]:
        
        
n = 5
t = 360
pop_size = 30

i = 2

#l = 3

total_prop = 0

for l in [3]:#range(1, n):
    
    # probability of having l out of n left at time t
    Q = np.zeros(shape=(n-l+1, n-l+1))
    for x in range(n-l):
        rate = scipy.misc.comb(n-x, 2) / (2 * pop_size)
        Q[x][x] = -rate
        Q[x][x+1] = rate
    prop_coalescences_n_to_l = scipy.linalg.expm(Q*t)[0][-1]
    prop_coalescences_n_to_l

    
    if i == 1 or l == 1:
        prop_ancestor_to_i = 1 # CHECK THAT THIS IS OK
    else:
        prop_ancestor_to_i = scipy.misc.comb(n-i-1, l-2) / scipy.misc.comb(n-1, l-1)

    assert prop_ancestor_to_i <= 1, prop_ancestor_to_i

    prob = prop_coalescences_n_to_l * prop_ancestor_to_i

    print(prop_coalescences_n_to_l, prop_ancestor_to_i)

    total_prop += prob
total_prop

IndentationError: expected an indented block (<ipython-input-15-ec3f8967074d>, line 18)

In [ ]:
Q

## Strategy

- We take demography from Yun Song.
- We make sure it is expected to the reduction non-African X/A from an african one of 0.66 reported in Mallick et al., and that it the theoretically predicted pi mathches the one observed in non-Africans. That works if we multiply each Ne in the "slim_demography" by 1.1.
- **The question is how much to reduce local Ne by**
- We simulate with Ne_autosome * 66/75
- In simulations we also use a reduced rec rate.

## Compute empirical X/A values for Africans and non-Africans

Compute X/A ratio for the males in our data set.

## Extract pi on X for Africans and non-Africans

In [ ]:
dist_afr = (pd.read_hdf(ag.steps_dir / 'male_dist_admix_masked_stores/male_dist_data_chrX_100kb.store')
            .loc[lambda df: (df.region_1 == 'Africa') & (df.region_2 == 'Africa')]
           )
dist_nonafr = (pd.read_hdf(ag.steps_dir / 'male_dist_admix_masked_stores/male_dist_data_chrX_100kb.store')
            .loc[lambda df: (df.region_1 != 'Africa') & (df.region_2 != 'Africa')]
           )
dist_nonafr.head()

In [ ]:
afr_mean_dist = dist_afr.dist.mean()
nonafr_mean_dist = dist_nonafr.dist.mean()

In [ ]:
afr_mean_dist, nonafr_mean_dist, nonafr_mean_dist / afr_mean_dist

## Extract likely pi relative to global pi in Africans

In [ ]:
lst = list()
for tup in extended_peak_regions_10.itertuples():
    lst.append(dist_afr.loc[(dist_afr.start >= tup.start_pos) & (dist_afr.end <= tup.end_pos)])
afr_dist_regions = pd.concat(lst)

lst = list()
for tup in extended_peak_regions_10.itertuples():
    lst.append(dist_nonafr.loc[(dist_nonafr.start >= tup.start_pos) & (dist_nonafr.end <= tup.end_pos)])
nonafr_dist_regions = pd.concat(lst)

In [ ]:
afr_mean_dist_in_regions = afr_dist_regions.dist.mean()
nonafr_mean_dist_in_regions = nonafr_dist_regions.dist.mean()

In [ ]:
afr_mean_dist_in_regions, afr_mean_dist, afr_mean_dist_in_regions / afr_mean_dist

In [ ]:
nonafr_mean_dist_in_regions, nonafr_mean_dist, nonafr_mean_dist_in_regions / nonafr_mean_dist

In [ ]:
nonafr_mean_dist_in_regions / afr_mean_dist_in_regions, nonafr_mean_dist / afr_mean_dist

## Compute demographies

In [ ]:
sim_generations = 200000 - 1
x_auto_fraction = 0.55

def years2gen(y):
    return int(1 + sim_generations - y / ag.gen_time)

years2gen(sim_generations * ag.gen_time), years2gen(0), sim_generations * ag.gen_time

In [ ]:
changes_year = [sim_generations * ag.gen_time, 250000, 150000, 100000, 70000, 50000, 35000, 25000, 15000, 6000]
changes_Ne = [20000, 12000, 6000, 4000, 3000, 4000, 6000, 12000, 20000, 100000]
sweep_years = [30000, 50000, 70000, 100000]

#changes_Ne = [(x*1.15) for x in changes_Ne]


In [ ]:
with sns.axes_style('ticks') :

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13,5), sharey=True)

    y = Series(list(reversed(changes_year)))
    n = Series(list(reversed(changes_Ne)))
    x = [j for i in zip(y.shift(), y) for j in i]
    y = [j for i in zip(n, n) for j in i]
    x[0] = 1000

    auto_demography = list(zip(reversed(x[1::2]), reversed(y[1::2])))

    color=next(ax1._get_lines.prop_cycler)['color']
    ax1.loglog(x, y, color=color)
    ax1.loglog(*list(zip(*auto_demography)), marker='o', color=color, linestyle='None', label='Autosomes');


    y = Series(list(reversed(changes_year)))
    n = Series(list(reversed(changes_Ne))) * x_auto_fraction
    x = [j for i in zip(y.shift(), y) for j in i]
    y = [j for i in zip(n, n) for j in i]
    x[0] = 1000

#     x_demography = [(y, n*x_auto_fraction) for (y, n) in auto_demography]

#     color=next(ax1._get_lines.prop_cycler)['color']
#     ax1.loglog(x, y, color=color)
#     ax1.loglog(*list(zip(*x_demography)), marker='o', color=color, linestyle='None', label='X');
#     ax1.set_ylim(1000, 120000)
#     ax1.set_xlim(1e3, 1e7) ;
#     ax1.set_xlabel('Years')
#     ax1.set_ylabel('Population size')
#     ax1.legend()

    [ax1.axvline(x, linestyle='dotted', color='black') for x in sweep_years]
    
    
    
    slim_demography = [(years2gen(y), int(n)) for (y, n) in auto_demography]  ## using auto
    g, n = list(zip(*slim_demography))
    ax2.loglog([sim_generations-x for x in g], n, marker='o', linestyle='None', 
               label='Slim pop. size change points\n(past to present)');
    ax2.set_ylim(1000, 120000)
    ax2.set_xlim(1e2, 2.2e5) ;
    ax2.set_xlabel('Generations')
    ax2.set_ylabel('Population size')
    ax2.legend()

    sns.despine()

We use **autosomal** sizes for Slim because it models the X chrom explicitly. To make the X/A ratio lower than 0.75 we scale the population sizes by (e.g. 0.66/0.75) when running the slim simulation.

Generation-Ne pairs for changeing population size in Slim simulation (for adding to `workflow.py`):

In [ ]:
slim_demography

Generations for sweeps in Slim simulations (for adding to `workflow.py`):

In [ ]:
sweep_generations = [years2gen(x) for x in sweep_years]
sweep_generations

In [ ]:
# sim_generations = 100000

# def years2gen(y):
#     return int(sim_generations - y / 29)
    
    
# def demography(pop_size, bottle_pop_size, bottle_start, bottle_end):
    
#     def bottleneck_N(t):
#         return int(round(t / (1/bottle_pop_size - (1-t)/pop_size)))

#     t = Series([1, 2/3, 1/3])
#     demog = DataFrame(dict(bottle_pop_size=t.map(bottleneck_N), 
#                                    bottle_time=t*(bottle_start - bottle_end)))
#     demog['bottle_start'] = round(bottle_start - (bottle_start - bottle_end - demog.bottle_time) / 2).astype(int)
#     demog['bottle_end'] = round(bottle_end + (bottle_start - bottle_end - demog.bottle_time) / 2).astype(int)
#     demog['pop_size'] = pop_size
#     return demog

# smallest_x_auto_fraction = 0.5
# pop_size = int(20000 * smallest_x_auto_fraction)
# bottle_pop_size = int(2000 * smallest_x_auto_fraction)
# bottle_start = 100000
# bottle_end = 30000

# demog = pd.concat([demography(10000, 1000, 100000, 30000),
#                   # demography(20000, 2000, 100000, 30000)
#                   ])
# demog

Print for loading into `workflow.py`:

In [ ]:
# list(zip(
#     list(demog.pop_size),
#     list(years2gen(x) for x in demog.bottle_start),
#     list(years2gen(x) for x in demog.bottle_end),
#     demog.bottle_pop_size))

## Pool-Nielsen computation

In [ ]:

epoques = list()
g, n = list(zip(*slim_demography))
gens, Ns = [sim_generations-x for x in g][::-1], n[::-1]
for i in range(len(gens)):
    if i == 0:
        epoques.append((gens[i], Ns[i]))
    elif i == len(gens) - 1:
        epoques.append((None, Ns[i]))    
    else:
        epoques.append((gens[i] - gens[i-1], Ns[i]))
epoques


In [ ]:
def exp_coal(g, N):
    return 2*N - (g * np.exp(-g/(2*N))) / (1 - np.exp(-g/(2*N)))

def epoch(demog, h, i):
    g, N = demog[i]
    N *= h
    if i == len(demog)-1:
        return 2*N
    return (1-np.exp(-g/(2*N))) * exp_coal(g, N) + np.exp(-g/(2*N)) * (g + epoch(demog, h, i+1))

A few sanity checks on the demography:

Make sure the demography produces the non-African X/A pi when we assume that African X/A pi reflects X/A Ne. Reading off from the plot in Mallick et al., the African X/A pi is 0.66 and the non-African one is 0.55.

In [ ]:
# epoques = [(x, y*1.15) for x, y in epoques]
# epoques

In [ ]:
epoch(epoques, 0.66, 0) / epoch(epoques, 1, 0)

That seems ok. We also want the actual values of non-African X pi to match the one produced by the demography.

In [ ]:

2 * 1.2e-8 * epoch(epoques, 0.66, 0), nonafr_mean_dist

In a neutral scenario, the demography should also produce the within-region pi of non-Africans from the within-region pi in Africans.




In [ ]:
2 * 1.2e-8 * epoch(epoques, 0.66*0.73, 0), nonafr_mean_dist_in_regions

That actually seems to be roughly the case. - Suggesting that a neutral scenario could indeed produce the observed pi values.

To see if it may also produce the ECHs, we need to perform the sumulations.

that the ratio in africa outside regions fit observed X/A in in non-africans using pool-nielsen. That serves as a sanity check that the demography model is ok.

In [ ]:
2 * 1.2e-8 * epoch(epoques, african_x_auto_ratio, 0), nonafr_mean_dist

That seems to fit what they find in Mallick et al.

Then if I take into account the lower Ne in region in africans, I get:

*(what is the lowest possible X/A ratio? 0.5?)*

In [ ]:
#epoques = [(1000, 1000), (1000, 1000), (1000, 1000), (1000, 1000), (None, 1000)]

african_x_auto_ratio = 0.66 * 0.73
pool_nielsen_ratio = epoch(epoques, african_x_auto_ratio, 0) / epoch(epoques, 1, 0)
african_x_auto_ratio, pool_nielsen_ratio

In [ ]:
2 * 1.2e-8 * epoch(epoques, african_x_auto_ratio, 0), nonafr_mean_dist_in_regions

How does that square with the empirical pi values and X/A values?

In [ ]:
x = np.linspace(0.001, 1, 100)
y = [epoch(epoques, h, 0) / epoch(epoques, 1, 0) for h in x]
plt.plot(x, y) ;
plt.axvline(african_x_auto_ratio, color='black', linestyle='dashed')
plt.axhline(pool_nielsen_ratio, color='black', linestyle='dashed') ;

## Lift over regions to hg38

In [ ]:
extended_peak_regions_10.head()

In [ ]:
print(extended_peak_regions_10.loc[lambda df: df.peak_prop_swept >= 0.5, 
                                   ['chrom', 'start_pos', 'end_pos']].to_string(index=False))

In [ ]:
lifted = '''
chrX	19281882	20581882	chrX:19300001-20600000	1
chrX	20481882	21681882	chrX:20500001-21700000	1
chrX	35681883	37080183	chrX:35700001-37100000	1
chrX	37340747	37940747	chrX:37200001-37800000	1
chrX	50057375	50057403	chrX:49400001-50300000	1
chrX	53373079	54873567	chrX:53400001-54900000	1
chrX	63280123	64680120	chrX:62500001-63900000	1
chrX	64580120	66080158	chrX:63800001-65300000	1
chrX	73080161	74380165	chrX:72300001-73600000	1
chrX	74480165	75680165	chrX:73700001-74900000	1
chrX	77448201	78544503	chrX:76700001-77800000	1
chrX	98945002	100145002	chrX:98200001-99400000	1
chrX	109856771	112356772	chrX:109100001-111600000	1
chrX	114365547	115465284	chrX:113600001-114700000	1
chrX	78550431	78550462	chrX:113600001-114700000	2
chrX	114761888	114761918	chrX:113600001-114700000	3
chrX	114761888	114761980	chrX:113600001-114700000	4
chr17	60552205	60552228	chrX:113600001-114700000	5
chrX	126866017	128566022	chrX:126000001-127700000	1
chrX	130366026	131066026	chrX:129500001-130200000	1
chrX	131865972	132665972	chrX:131000001-131800000	1
'''

In [ ]:
print(extended_peak_regions_10[['chrom', 'start_pos', 'end_pos']].to_string(index=False))

Used online liftover to convert to hg38 allowing, checking box to allow multiple output regions. Pasted in below:

In [ ]:
lifted = '''
chrX	10781881	11681880	chrX:10800001-11700000	1
chrX	14181878	15181878	chrX:14200001-15200000	1
chrX	19281882	20581882	chrX:19300001-20600000	1
chrX	20481882	21681882	chrX:20500001-21700000	1
chrX	35681883	37080183	chrX:35700001-37100000	1
chrX	37340747	37940747	chrX:37200001-37800000	1
chrX	43140751	43740753	chrX:43000001-43600000	1
chrX	50057375	50057403	chrX:49400001-50300000	1
chrX	50957000	51657066	chrX:50700001-51400000	1
chr2	33174181	33174306	chrX:50700001-51400000	2
chrX	53373079	54873567	chrX:53400001-54900000	1
chrX	54973567	56273567	chrX:55000001-56300000	1
chrX	63280123	64680120	chrX:62500001-63900000	1
chrX	64580120	66080158	chrX:63800001-65300000	1
chrX	67280158	68180158	chrX:66500001-67400000	1
chrX	68080158	68680158	chrX:67300001-67900000	1
chrX	73080161	74380165	chrX:72300001-73600000	1
chrX	74480165	75680165	chrX:73700001-74900000	1
chrX	77448201	78544503	chrX:76700001-77800000	1
chrX	98945002	100145002	chrX:98200001-99400000	1
chrX	101545013	102145028	chrX:100800001-101400000	1
chrX	105255317	106056009	chrX:104500001-105300000	1
chrX	107056770	108256770	chrX:106300001-107500000	1
chrX	109856771	112356772	chrX:109100001-111600000	1
chrX	114365547	115465284	chrX:113600001-114700000	1
chrX	78550431	78550462	chrX:113600001-114700000	2
chrX	114761888	114761918	chrX:113600001-114700000	3
chrX	114761888	114761980	chrX:113600001-114700000	4
chr17	60552205	60552228	chrX:113600001-114700000	5
chrX	126866017	128566022	chrX:126000001-127700000	1
chrX	130366026	131066026	chrX:129500001-130200000	1
chrX	131865972	132665972	chrX:131000001-131800000	1
chrX	133165972	134465970	chrX:132300001-133600000	1
chrX	154671726	155270710	chrX:153900001-154500000	1
'''

## Extract relevant recombination rate

In [ ]:
records = list()
for row in lifted.strip().split('\n'):
    records.append([x.isdigit() and int(x) or x for x in row.split()])
extended_peak_regions_10_hg38 = pd.DataFrame.from_records(records, 
                                    columns=['chrom', 'start', 'end', 'segment', 'piece'])
extended_peak_regions_10_hg38

In [ ]:
decode_maternal = pd.read_table(ag.data_dir / 'decode_hg38_maternal.tsv', comment='#')
decode_maternal.head()

Mean rate for entire chromosome.

In [ ]:
decode_maternal_chrX = decode_maternal.loc[decode_maternal['Chr'] == 'chrX']
mean_maternal_rate = np.average(decode_maternal_chrX.cMperMb, weights=decode_maternal_chrX.End-decode_maternal_chrX.Begin)
mean_maternal_rate

In [ ]:
lst = list()
for tup in extended_peak_regions_10_hg38.itertuples():
    lst.append(decode_maternal_chrX.loc[(decode_maternal_chrX.Begin >= tup.start) & \
                                         (decode_maternal_chrX.End <= tup.end)])
df = pd.concat(lst).assign(length=lambda df: df.End-df.Begin)

In [ ]:
# x = 0
# for tup in df.itertuples():
#     x += tup.length / tup.cMperMb
# df.cMperMb.sum() / x

In [ ]:
region_rates = list()
for tup in extended_peak_regions_10_hg38.itertuples():
    df = decode_maternal_chrX.loc[(decode_maternal_chrX.Begin >= tup.start) & \
                                         (decode_maternal_chrX.End <= tup.end)]
    try:
        print(np.average(df.cMperMb, weights=df.End-df.Begin), (df.End-df.Begin).sum())
    except:
        print('no data')
    region_rates.append(df.cMperMb.mean())
#region_rates

Mean per-generation recombination rate for all the regions:

In [ ]:
df_list = list()
for tup in extended_peak_regions_10_hg38.itertuples():
    df_list.append(decode_maternal_chrX.loc[(decode_maternal_chrX.Begin >= tup.start) & (decode_maternal_chrX.End <= tup.end)])
df = pd.concat(df_list)

In [ ]:
extended_peak_regions_10_mean_rate = np.average(df.cMperMb, weights=df.End-df.Begin)
extended_peak_regions_10_mean_rate

Mean per-meiosis recombination rate for all the regions:

In [ ]:
extended_peak_regions_10_mean_rate * 3 / 2

## Load data for sweeps called on simulations

In [ ]:
prop_swept_data = pd.read_hdf(str(ag.steps_dir / 'slim' / 'slim_summary.hdf'))
prop_swept_data['prop_swept'] = prop_swept_data.nr_swept / prop_swept_data.total
prop_swept_data.head()

In [ ]:
records = list()
for demog, x_auto_ratio, rec_rate_per_gen, chrom, sweep_type, sweep_start in prop_swept_data.simulation.str.split('_'):
    records.append([demog, int(x_auto_ratio)/100, int(rec_rate_per_gen)/1e-12, chrom, sweep_type, int(sweep_start)])

df = pd.DataFrame.from_records(records, 
                      columns=['demography', 'x_auto_ratio', 'rec_rate_per_gen', 
                               'chrom', 'sweep_type', 'sweep_start'], 
                              )

# df = pd.DataFrame.from_records(records, 
#                       columns=['sweep_type', 'N', 'bottle_start', 'bottle_end', 'bottle_width', 'sweep_start'], 
#                               )
# df['bottle_width'] = df.bottle_end - df.bottle_start
prop_swept_data = prop_swept_data.merge(df, left_index=True, right_index=True)
prop_swept_data.head()

In [ ]:
def facet_plot(start, end, prop_swept, **kwargs):
    x = [j for i in zip(start, end) for j in i]
    y = [j for i in zip(prop_swept, prop_swept) for j in i]      
    plt.plot(x, y, color='black', linewidth=0.2)
#    plt.plot(x, y, linewidth=0.2, **kwargs)
    plt.ylim((0, 1.1))
    plt.fill_between(x, 0, y, color='gray', alpha=0.05)


with sns.axes_style('whitegrid'):
#     g = sns.FacetGrid(prop_swept_data, col='simulation', row='selection_coef', 
    g = sns.FacetGrid(prop_swept_data, col='selection_coef', row='simulation', 
                      hue='replication', sharex=True, sharey=True, margin_titles=True,
                     height=2, aspect=5)
    g.map(facet_plot, 'start', 'end', 'prop_swept')

    for ax in g.axes.flat:
        plt.setp(ax.texts, text="") # remove the original texts, important to do this before setting titles
    g.set_titles(row_template = '{row_name}', col_template = '{col_name}')

    for ax in g.axes.flat:
        plt.setp(ax.texts, rotation="0") # remove the original texts, important to do this before setting titles



In [ ]:
with sns.axes_style('whitegrid'):
#     g = sns.FacetGrid(prop_swept_data, col='simulation', row='selection_coef', 
    g = sns.FacetGrid(prop_swept_data.loc[prop_swept_data.simulation.str.startswith('standard_40_4650')],
                      col='selection_coef', row='simulation', 
                      hue='replication', sharex=True, sharey=True, margin_titles=True,
                     height=3, aspect=5)
    g.map(facet_plot, 'start', 'end', 'prop_swept')

    for ax in g.axes.flat:
        plt.setp(ax.texts, text="") # remove the original texts, important to do this before setting titles
    g.set_titles(row_template = '{row_name}', col_template = '{col_name}')

    for ax in g.axes.flat:
        plt.setp(ax.texts, rotation="0") # remove the original texts, important to do this before setting titles



In [ ]:
prop_swept_data.simulation.unique()

In [ ]:
def facet_plot(start, end, prop_swept, **kwargs):
    x = [j for i in zip(start, end) for j in i]
    y = [j for i in zip(prop_swept, prop_swept) for j in i]      
    plt.plot(x, y, color='black', linewidth=0.2)
#    plt.plot(x, y, linewidth=0.2, **kwargs)
    plt.ylim((0, 1))
    plt.fill_between(x, 0, y, color='gray', alpha=0.5)

with sns.axes_style('whitegrid'):
#     g = sns.FacetGrid(prop_swept_data, col='simulation', row='selection_coef', 
    g = sns.FacetGrid(prop_swept_data.loc[prop_swept_data.simulation.isin(['standard_55_4650_X_nosweep_98275',
                                                                          'standard_40_4650_X_nosweep_98275',
                                                                          'standard_48_4650_X_nosweep_98275',
                                                                          'standard_66_4650_X_nosweep_98275'])],
                      col='replication', row='simulation', sharex=True, sharey=True, margin_titles=True,
                     height=4, aspect=0.5)
    g.map(facet_plot, 'start', 'end', 'prop_swept')

    for ax in g.axes.flat:
        plt.setp(ax.texts, text="") # remove the original texts, important to do this before setting titles
    g.set_titles(row_template = '{row_name}', col_template = '{col_name}')

    for ax in g.axes.flat:
        plt.setp(ax.texts, rotation="0") # remove the original texts, important to do this before setting titles



In [ ]:
df = pd.read_hdf(str(ag.steps_dir / 'slim/simulations/standard/66/4650/X/nosweep/98275/10/standard_66_4650_X_nosweep_98275_10_0.hdf'))

plt.hist(df.dist, bins=100, alpha=0.5) ;
plt.axvline(5e-5, color='orange', linestyle='dashed') ;


In [ ]:
df = pd.read_hdf(str(ag.steps_dir / 'slim/simulations/standard/48/4650/X/nosweep/98275/10/standard_48_4650_X_nosweep_98275_10_0.hdf'))

plt.hist(df.dist, bins=100, alpha=0.5) ;
plt.axvline(5e-5, color='orange', linestyle='dashed');


In [ ]:
# s = 'standard_40_4650_X_nosweep_98275_{}'.format(20)
# f_name_tmpl = '/home/kmt/simons/faststorage/people/kmt/steps/slim/simulations/' + s.replace('_', '/') + '/{}_{{}}.hdf'.format(s)
# df = pd.read_hdf(f_name_tmpl.format(0))
# plt.hist(df.dist, bins=200, alpha=0.5) ;

# s = 'standard_40_4650_X_nosweep_98275_{}'.format(20)
# f_name_tmpl = '/home/kmt/simons/faststorage/people/kmt/steps/slim/simulations/' + s.replace('_', '/') + '/{}_{{}}.hdf'.format(s)
# df = pd.read_hdf(f_name_tmpl.format(0))
# plt.hist(df.dist, bins=200, alpha=0.5) ;

In [ ]:
# s = 'nosweep_10000_96551_98965_10000_96206_{}'.format(20)
# f_name_tmpl = '/home/kmt/simons/faststorage/people/kmt/steps/slim/simulations/' + s.replace('_', '/') + '/{}_{{}}.hdf'.format(s)
# df = pd.read_hdf(f_name_tmpl.format(0))
# plt.hist(df.dist, bins=200, alpha=0.5) ;

# s = 'nosweep_10000_96551_98965_1000_96206_{}'.format(20)
# f_name_tmpl = '/home/kmt/simons/faststorage/people/kmt/steps/slim/simulations/' + s.replace('_', '/') + '/{}_{{}}.hdf'.format(s)
# df = pd.read_hdf(f_name_tmpl.format(0))
# plt.hist(df.dist, bins=200, alpha=0.5) ;

# s = 'nosweep_10000_96954_98563_690_98275_{}'.format(20)
# f_name_tmpl = '/home/kmt/simons/faststorage/people/kmt/steps/slim/simulations/' + s.replace('_', '/') + '/{}_{{}}.hdf'.format(s)
# df = pd.read_hdf(f_name_tmpl.format(0))
# plt.hist(df.dist, bins=200, alpha=0.5) ;

# s = 'nosweep_10000_97356_98160_357_99310_{}'.format(20)
# f_name_tmpl = '/home/kmt/simons/faststorage/people/kmt/steps/slim/simulations/' + s.replace('_', '/') + '/{}_{{}}.hdf'.format(s)
# df = pd.read_hdf(f_name_tmpl.format(0))
# plt.hist(df.dist, bins=100, alpha=0.5) ;


In [ ]:


s = 'nosweep_10000_3448_5861_10000_5171_{}'.format(20)
f_name_tmpl = '/home/kmt/simons/faststorage/people/kmt/steps/slim/simulations/' + s.replace('_', '/') + '/{}_{{}}.hdf'.format(s)
df = pd.read_hdf(f_name_tmpl.format(0))
plt.hist(df.dist, bins=100, alpha=0.5) ;


In [ ]:
s = 'nosweep_10000_3447_5861_1000_3102_{}'.format(20)
f_name_tmpl = '/home/kmt/simons/faststorage/people/kmt/steps/slim/simulations/' + s.replace('_', '/') + '/{}_{{}}.hdf'.format(s)
df = pd.read_hdf(f_name_tmpl.format(0))
plt.hist(df.dist, bins=200, alpha=0.5) ;

s = 'nosweep_10000_3448_5861_10000_3102_{}'.format(20)
f_name_tmpl = '/home/kmt/simons/faststorage/people/kmt/steps/slim/simulations/' + s.replace('_', '/') + '/{}_{{}}.hdf'.format(s)
df = pd.read_hdf(f_name_tmpl.format(0))
plt.hist(df.dist, bins=200, alpha=0.5) ;


In [ ]:
s = 'nosweep_10000_3447_5861_1000_3102_{}'.format(20)
f_name_tmpl = '/home/kmt/simons/faststorage/people/kmt/steps/slim/simulations/' + s.replace('_', '/') + '/{}_{{}}.hdf'.format(s)
df = pd.read_hdf(f_name_tmpl.format(0))
plt.hist(df.dist, bins=100) ;

In [ ]:
plt.figure(figsize=(10,6))
for i in range(10):
    plt.subplot(2, 5, i+1)
    df = pd.read_hdf(f_name_tmpl.format(i))
    plt.hist(df.dist, bins=100)
    plt.axvline(5e-5, color='orange', zorder=0)

In [ ]:
import pyslim, msprime, random
seed = 7
mutation_rate = 1.25e-8
sample_size = 100

trees = pyslim.load(f_name_tmpl.format(0).replace('.hdf', '.trees'))

# Overlay neutral mutations and make bit arrays for each SNP:
sample_ids = sorted(random.sample(list(trees.samples()), sample_size))

trees_sample = trees.simplify(samples=sample_ids, filter_zero_mutation_sites=True, filter_individuals=True)
mutated_trees = msprime.mutate(trees_sample, rate=mutation_rate, random_seed=seed)
# mutated.dump("./sweep_overlaid.trees") 
print(mutated_trees.pairwise_diversity() / mutated_trees.sequence_length)


ts = mutated_trees
# Measure the tree height at each base position
height_for_pos = np.zeros(int(ts.sequence_length))
for tree in ts.trees():
    mean_height = np.mean([tree.time(root) for root in tree.roots])
    left, right = map(int, tree.interval)
    height_for_pos[left: right] = mean_height
height_for_pos
plt.figure(figsize=(10,3))
plt.plot(height_for_pos, linewidth=0.5, color='black') ;


trees = pyslim.load(f_name_tmpl.format(1).replace('.hdf', '.trees'))

# Overlay neutral mutations and make bit arrays for each SNP:
sample_ids = sorted(random.sample(list(trees.samples()), sample_size))

trees_sample = trees.simplify(samples=sample_ids, filter_zero_mutation_sites=True, filter_individuals=True)
mutated_trees = msprime.mutate(trees_sample, rate=mutation_rate, random_seed=seed)
# mutated.dump("./sweep_overlaid.trees") 
print(mutated_trees.pairwise_diversity() / mutated_trees.sequence_length)


ts = mutated_trees
# Measure the tree height at each base position
height_for_pos = np.zeros(int(ts.sequence_length))
for tree in ts.trees():
    mean_height = np.mean([tree.time(root) for root in tree.roots])
    left, right = map(int, tree.interval)
    height_for_pos[left: right] = mean_height
height_for_pos

plt.plot(height_for_pos, linewidth=0.5, color='red') ;







## Run slim testing to get pairwise distances for testing purposes

In [ ]:
from itertools import combinations
import argparse
import random
import subprocess
import re, os, sys
import tempfile
import numpy as np
import pandas as pd
from pandas import DataFrame
import msprime, pyslim

trees_file = "/home/kmt/simons/faststorage/people/kmt/test_slim.trees"

slurm_script = r'''
initialize() {
	initializeTreeSeq();
	initializeMutationRate(0);
	initializeMutationType("m1", 0.5, "f", 0.0);
	initializeGenomicElementType("g1", m1, 1.0);
	initializeGenomicElement(g1, 0, 10e6-1);
	initializeRecombinationRate(1e-8);
}
1 {
	defineConstant("simID", getSeed());
	sim.addSubpop("p1", 10000);
}
50000 {
    sim.treeSeqOutput("OUTFILE");
	sim.simulationFinished();
}
'''.replace('OUTFILE', trees_file)

print(slurm_script)
random.seed(7)

window_size = 100000

# write slim script file with the right output name
slurm_script_file = tempfile.NamedTemporaryFile(mode='w', delete=False)
slurm_script_file.write(slurm_script)
slurm_script_file.close()

# run slim
cmd = '/home/kmt/simons/faststorage/people/kmt/slim {}'.format(slurm_script_file.name)
p = subprocess.Popen(cmd.split(), 
    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = p.communicate()
print(stdout.decode())
print(stderr.decode())

In [ ]:
mutationrate = 4.3e-10
generationtime = 29

# load trees from slim
ts = pyslim.load(trees_file)

# overlay mutations
mutated_ts = msprime.mutate(ts, rate=mutationrate*generationtime, random_seed=2, keep=True)

In [ ]:
ts = mutated_trees
# Measure the tree height at each base position
height_for_pos = np.zeros(int(ts.sequence_length))
for tree in ts.trees():
    mean_height = np.mean([tree.time(root) for root in tree.roots])
    left, right = map(int, tree.interval)
    height_for_pos[left: right] = mean_height
height_for_pos
plt.figure(figsize=(10,3))
plt.plot(height_for_pos, linewidth=0.5, color='black') ;

height_for_pos.mean()

In [ ]:
samples = 10

# random indexes for samples
sample_idx = set(random.sample(range(ts.num_individuals), samples))
assert len(sample_idx) == samples

In [ ]:
# get the corresponding sample haplotypes
sample = list()
for i, hap in enumerate(mutated_ts.haplotypes()):
    if i in sample_idx:
        sample.append(hap)        

In [ ]:
        
# get the positions of each segregating site
positions = [site.position for site in mutated_ts.sites()]  

# make table with sampled haplotypes
table = np.array([list(map(np.int8, hap)) for hap in sample]).transpose()

In [ ]:
# turn table into dataframe with positions
df = DataFrame(table, dtype='int8')
df['pos'] = positions

In [ ]:
# add a row with zeros for the start of each window so there is at least
# one row in each window
zeros = dict((x, 0) for x in range(samples))
extra_df = pd.DataFrame({'pos': range(0, int(mutated_ts.sequence_length), window_size), **zeros})
df = df.append(extra_df)

In [ ]:
# make a start column grouping all rows in same window
df['start'] = ((df.pos // window_size) * window_size).astype('uint32')
df.drop('pos', axis=1, inplace=True)
df.set_index('start', inplace=True)

def pw_dist(df):
    "computes differences bewteen all pairs in a window"
    pairs = list(combinations(df.columns, 2))
    site_diffs = [np.bitwise_xor(df[p[0]], df[p[1]]) for p in pairs]
    return pd.concat(site_diffs, axis=1, keys=pairs).sum()

# make a dataframe with distance for each pair
pw_dist_df = (
    df
    .groupby('start')
    .apply(pw_dist)
    .reset_index()
    .melt(id_vars=['start'], var_name=['indiv_1', 'indiv_2'], value_name='dist')
    )

In [ ]:
# compute proper distance as number of diffs divided by window size
pw_dist_df['dist'] /= window_size

# add end column
pw_dist_df.insert(loc=1, column='end', value=pw_dist_df.start + window_size)

# convert indiv labels from object to int and and write hdf
pw_dist_df['indiv_1'] = pw_dist_df['indiv_1'].astype('uint16')
pw_dist_df['indiv_2'] = pw_dist_df['indiv_2'].astype('uint16')
#pw_dist_df.to_hdf(args.hdf_file, 'df', format='table', mode='w')

In [ ]:
plt.hist(pw_dist_df.dist) 
plt.axvline(5e-5, color='orange') ;